In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset, Subset
from PIL import Image
import pandas as pd

img_dir = './data/images'
#train_binarized is a binary list of labels for each image
csv_file = './data//train_binarized.csv'

# Createa data loader from csv labels and images 
class ImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_path = f"{self.img_dir}/{idx}.jpg"
        image = Image.open(img_path).convert('RGB')
        labels = torch.tensor(self.labels_df.iloc[idx].values, dtype=torch.float32)
        
        if self.transform:
            image = self.transform(image)
        
        return image, labels


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

full_dataset = ImageDataset(csv_file=csv_file, img_dir=img_dir, transform=transform)

train_loader = DataLoader(full_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(full_dataset, batch_size=10, shuffle=False)

# Load a pre-trained ResNet model and modify it
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    #18 output features for multi-labels
    nn.Linear(num_features, 18), 
    nn.Sigmoid()
)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")



train_model(model, train_loader, criterion, optimizer, num_epochs=10)



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [33]:
import torch
from torchvision import transforms
from PIL import Image
from torchvision.models import resnet50
import torch.nn as nn

# Model weights after training, have not been trained yet
# model.load_state_dict(torch.load('path_to_model_weights.pth'))
model.eval()  

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization used for ResNet
])

def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)  
    image = image.unsqueeze(0) 

    with torch.no_grad():  
        output = model(image)
        predictions = (output > 0.3).int()  

    return predictions.numpy()  

# Example usage:
image_path = './data/images/19.jpg'
prediction = predict_image(image_path)
print(prediction)


[[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]
